In [36]:
import os
import psycopg2
import yaml
from sqlalchemy import create_engine
import pandas as pd

# from database_utils import DatabaseConnector

In [13]:
class DatabaseConnector:
    def read_db_creds(self, filename='db_creds.yaml'):
        with open(filename, 'r') as file:
            db_creds = yaml.safe_load(file)
        return db_creds

    def init_db_engine(self):
        creds = self.read_db_creds()
        db_uri = f"postgresql://{creds['RDS_USER']}:{creds['RDS_PASSWORD']}@{creds['RDS_HOST']}:{creds['RDS_PORT']}/{creds['RDS_DATABASE']}"
        engine = create_engine(db_uri)
        return engine

    def list_db_tables(self):
        engine = self.init_db_engine()
        with engine.connect() as conn:
            tables = conn.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
            table_names = [table[0] for table in tables]
        return table_names


In [14]:
connector = DatabaseConnector()

In [15]:
tables = connector.list_db_tables()

C:\Users\User\AppData\Local\Temp\ipykernel_16400\3217606405.py:16: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  tables = conn.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")


In [16]:
tables

['legacy_store_details', 'legacy_users', 'orders_table']

In [41]:
import os
import psycopg2
import yaml
from sqlalchemy import create_engine
import pandas as pd
import tabula


class DataExtractor:
    def __init__(self):
        self.connector = DatabaseConnector()

    def read_rds_table(self, table_name):
        engine = self.connector.init_db_engine()
        with engine.connect() as connection:
            query = f"SELECT * FROM {table_name}"
            df = pd.read_sql(query, connection)
            return df
        
    def retrieve_pdf_data(self, pdf_link):
        pdf_data = tabula.read_pdf(pdf_link, pages='all')
        df = pd.concat(pdf_data, ignore_index=True)
        return df



In [18]:
extractor = DataExtractor()

In [19]:
user_data = extractor.read_rds_table('legacy_users')

In [20]:
user_data

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579
...,...,...,...,...,...,...,...,...,...,...,...,...
15315,14913,Stephen,Jenkins,1943-08-09,"Thornton, Carroll and Newman",s.jenkins@smith.com,Studio 41I\nJones lodge\nOliviaborough\nE8 3DU,United Kingdom,GB,+44(0)292018946,2016-04-15,2bd3a12f-a92d-4cdd-b99c-fc70572db302
15316,14994,Stephen,Smith,1948-08-20,Robinson-Harris,s.smith@smith.com,530 Young parkway\nMillsfurt\nL4G 7NX,United Kingdom,GB,+44(0)1144960977,2020-07-20,d234c04b-c07c-46a5-a902-526f91478ecc
15317,15012,Stephen,Losekann,1940-10-09,Rosenow,s.losekann@smith.com,Viviane-Fritsch-Straße 3/5\n15064 Bad Liebenwerda,Germany,DE,02984 08192,2021-03-07,1a0a8b7b-7c17-42d8-a946-8a85d5495651
15318,15269,Stephen,Rivera,1952-06-04,"Taylor, Fry and Jones",s.rivera@smith.com,"660 Ross Falls Suite 357\nAnthonymouth, MA 09610",United States,US,239.711.3836,2011-01-03,187fe06e-bd5f-4381-af2f-d7ac37ca7572


In [25]:
import pandas as pd
from datetime import datetime

class DataCleaning:
    def clean_user_data(self, user_data):
        # Drop rows with NULL values
        user_data = user_data.dropna()

        # Convert date columns to datetime format
        date_columns = ['date_of_birth', 'join_date']
        for column in date_columns:
            user_data[column] = pd.to_datetime(user_data[column], errors='coerce')

        # Drop rows with invalid date values
        user_data = user_data.dropna(subset=date_columns)

        # Convert columns to appropriate data types
        user_data['user_uuid'] = user_data['user_uuid'].astype(str)
        user_data['country'] = user_data['country'].astype(str)
        user_data['email_address'] = user_data['email_address'].astype(str)

        return user_data


In [26]:
cleaner = DataCleaning()

In [27]:
cleaned_data = cleaner.clean_user_data(user_data)

In [28]:
cleaned_data

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579
...,...,...,...,...,...,...,...,...,...,...,...,...
15315,14913,Stephen,Jenkins,1943-08-09,"Thornton, Carroll and Newman",s.jenkins@smith.com,Studio 41I\nJones lodge\nOliviaborough\nE8 3DU,United Kingdom,GB,+44(0)292018946,2016-04-15,2bd3a12f-a92d-4cdd-b99c-fc70572db302
15316,14994,Stephen,Smith,1948-08-20,Robinson-Harris,s.smith@smith.com,530 Young parkway\nMillsfurt\nL4G 7NX,United Kingdom,GB,+44(0)1144960977,2020-07-20,d234c04b-c07c-46a5-a902-526f91478ecc
15317,15012,Stephen,Losekann,1940-10-09,Rosenow,s.losekann@smith.com,Viviane-Fritsch-Straße 3/5\n15064 Bad Liebenwerda,Germany,DE,02984 08192,2021-03-07,1a0a8b7b-7c17-42d8-a946-8a85d5495651
15318,15269,Stephen,Rivera,1952-06-04,"Taylor, Fry and Jones",s.rivera@smith.com,"660 Ross Falls Suite 357\nAnthonymouth, MA 09610",United States,US,239.711.3836,2011-01-03,187fe06e-bd5f-4381-af2f-d7ac37ca7572


In [37]:
class DatabaseConnector:
    def __init__(self):
        self.engine = self.init_db_engine()
        self.upload_engine = self.init_upload_engine()

    def read_db_creds(self, filename='db_creds.yaml'):
        with open(filename, 'r') as file:
            db_creds = yaml.safe_load(file)
        return db_creds

    def init_db_engine(self):
        creds = self.read_db_creds()
        db_uri = f"postgresql://{creds['RDS_USER']}:{creds['RDS_PASSWORD']}@{creds['RDS_HOST']}:{creds['RDS_PORT']}/{creds['RDS_DATABASE']}"
        engine = create_engine(db_uri)
        return engine

    def list_db_tables(self):
        with self.engine.connect() as conn:
            tables = conn.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
            table_names = [table[0] for table in tables]
        return table_names
    
    def init_upload_engine(self):
        creds = self.read_db_creds()
        db_uri = f"postgresql://{creds['USER']}:{creds['PASSWORD']}@{creds['HOST']}:{creds['PORT']}/{creds['DATABASE']}"
        engine = create_engine(db_uri)
        return engine


    def upload_to_db(self, df, table_name):
        df.to_sql(table_name, self.upload_engine, if_exists='replace', index=False)

In [38]:
connector = DatabaseConnector()

In [39]:
connector.upload_to_db(cleaned_data, 'dim_users')

In [42]:
extractor_pdf = DataExtractor()

In [45]:
pdf_data = extractor_pdf.retrieve_pdf_data("https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf")

JavaNotFoundError: `java` command is not found from this Python process.Please ensure Java is installed and PATH is set for `java`

In [51]:
card_data = pd.read_csv(r"C:\Users\User\Documents\AiCore\card_details.csv")

In [52]:
card_data

,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,25/11/2015
1,349624180933183,10/23,American Express,18/06/2001
2,3529023891650490,06/23,JCB 16 digit,26/12/2000
3,213142929492281,09/27,JCB 15 digit,12/02/2011
4,502067329974,10/25,Maestro,13/03/1997
...,...,...,...,...
15304,180036921556789,12/28,JCB 15 digit,06/06/1997
15305,180018030448512,11/24,JCB 15 digit,16/06/2004
15306,3569953313547220,04/24,JCB 16 digit,05/02/2020
15307,4444521712606810,06/27,VISA 16 digit,16/06/2008


In [56]:
import pandas as pd
from datetime import datetime

class DataCleaning:
    def clean_user_data(self, user_data):
        # Drop rows with NULL values
        user_data = user_data.dropna()

        # Convert date columns to datetime format
        date_columns = ['date_of_birth', 'join_date']
        for column in date_columns:
            user_data[column] = pd.to_datetime(user_data[column], errors='coerce')

        # Drop rows with invalid date values
        user_data = user_data.dropna(subset=date_columns)

        # Convert columns to appropriate data types
        user_data['user_uuid'] = user_data['user_uuid'].astype(str)
        user_data['country'] = user_data['country'].astype(str)
        user_data['email'] = user_data['email'].astype(str)

        user_data = user_data.drop(columns=['index'])

        return user_data
    
    def clean_card_data(self, card_data):
        # Drop rows with NULL values
        card_data = card_data.dropna()

        # Remove leading '???' in card_number and convert to integer
        card_data['card_number'] = card_data['card_number'].str.replace(r'\?\?\?', '').astype(int)

        # Convert expiry_date from mm/yy to dd/mm/yyyy
        card_data['expiry_date'] = pd.to_datetime(card_data['expiry_date'], format='%m/%y', errors='coerce')
        card_data['expiry_date'] = card_data['expiry_date'].dt.strftime('%d/%m/%Y')

        # Convert card_provider to string
        card_data['card_provider'] = card_data['card_provider'].astype(str)

        return card_data



In [57]:
card_cleaner = DataCleaning()

In [58]:
clean_card = card_cleaner.clean_card_data(card_data)

C:\Users\User\AppData\Local\Temp\ipykernel_16400\902786966.py:31: FutureWarning: The default value of regex will change from True to False in a future version.
  card_data['card_number'] = card_data['card_number'].str.replace(r'\?\?\?', '').astype(int)


OverflowError: Python int too large to convert to C long